# Serial time dinamics
Estudiar efectos del tiempo serial en la dinámica:
1. Manteniendo el tiempo serial y el R0 (variando el beta), variar sigma y gama para ver cómo responde la dinámica del virus. 
2. Manteniendo tiempo serial, gamma y beta, ver el efecto de variar sigma (1/tiempo de incubación) 

In [ ]:
import multiprocessing
from joblib import Parallel, delayed
import sys
from pathlib import Path
sys.path.insert(1, '../../src/SEIR/')
sys.path.insert(1, '../../src/SEIRHVD/')
sys.path.insert(1, '../../src/utils/')

import numpy as np
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

from class_SEIR2 import SEIR

# Parámetros
### Tiempo de incubación  
$T_{inc} = \frac{1}{\sigma}$
### Tiempo de infecciosidad  
$T_{inf} = \frac{1}{\gamma}$
### Tiempo Serial  
$T_s = T_{inc} + \frac{T_{inf}}{2}$

### Numero reproductivo básico  
$R_0 = \frac{\beta}{\gamma}$

# Ecuaciones
$\dot{S} = -\alpha\beta \frac{SI}{N} $   
$\dot{E} = \alpha\beta \frac{SI}{N} - \sigma E$   
$\dot{I} =  \sigma E -\gamma I$   
$\dot{R} = \gamma I$   

# Estudio efectos del tiempo de incubación
En estas simulaciones se simula para distintos tiempos de incubación manteniendo todas las otras variables fijas. Al variar el tiempo de incubación y mantener el tiempo de infecciosidad constante, el tiempo serial también variará

In [ ]:
# Simulation time
tsim = 1000

# Parameters
R0 = 1.2


step = 0.5

T_inf = 6
T_inc = np.array(np.arange(1,6,0.5))
T_s = T_inc + T_inf/2
gamma = 1/T_inf
sigma = 1/T_inc

beta = R0*gamma 

In [ ]:
#Initial Conditions
mu = R0
population = 100000
I = 1
E = I*mu
R = 0
S = population - I - E - R

In [ ]:
def alpha(t):
    return 1

In [ ]:
i = 0
sim = [SEIR(tsim=tsim,alpha=alpha,beta=beta,mu=mu,sigma=sigma[i],gamma=gamma,I0=I,I_ac0=I,I_d0=0,population=population) for i in range(len(T_inc))]


### Simulate

In [ ]:
def simulate(simulation,i,tsim):
    simulation[i].integr()
    return simulation[i]

In [ ]:
num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sim,i,tsim) for i in range(len(sim)))
sim = sims
print('ready')

### Results

In [ ]:
for i in range(len(T_inc)):
    #plt.plot(sim[i].t,sim[i].S,label='(S) T_inc: '+str(T_inc[i]))
    #plt.plot(sim[i].t,sim[i].E,label='(E) T_inc: '+str(T_inc[i]))
    plt.plot(sim[i].t,sim[i].I,label='(I) T_inc: '+str(T_inc[i]))
    #plt.plot(sim[i].t,sim[i].R,label='(R) T_inf: '+str(T_inc[i]))
plt.title('Incubation time variation')
plt.legend(loc=0)
plt.plot()

In [ ]:
peak = [sim[i].peak for i in range(len(T_inc))]
peak_t = [sim[i].peak_t for i in range(len(T_inc))]

In [ ]:
plt.plot(T_inc,peak,label='Peak size vs Infectious period')
plt.title('Peak size vs Incubation time')
plt.legend(loc=0)
plt.plot()

In [ ]:
plt.plot(T_inc,peak_t,label='Peak day')
plt.title('Peak day vs Incubation time')
plt.legend(loc=0)
plt.plot()

In [ ]:
R_final = [sim[i].R[-1] for i in range(len(T_inc))]

In [ ]:
plt.plot(T_inc,R_final,label='Infectados totales')
plt.title('R al final vs Incubation time')
plt.legend(loc=0)
plt.plot()

# Tiempo serial y R fijo - Variacion sigma y gamma
Para mantener el R fijo, al variar gamama variamos beta proporcionalmente.

In [ ]:
# Simulation time
tsim = 1000

# Parameters
R0 = 1.2
T_s = 8

step = 0.5
# gamma
T_inf = np.array(np.arange(1,7+step,step))
# sigma
T_inc = T_s-T_inf/1
gamma = 1/T_inf
sigma = 1/T_inc

beta = R0*gamma 

In [ ]:
#Initial Conditions
mu = 0#R0
population = 100000
I = 1
E = I*mu
R = 0
S = population - I - E - R

In [ ]:
def alpha(t):
    return 1

In [ ]:
i = 0
sim = [SEIR(tsim=tsim,alpha=alpha,beta=beta[i],mu=mu,sigma=sigma[i],gamma=gamma[i],I0=I,I_ac0=I,I_d0=0,population=population) for i in range(len(T_inf))]


### Simulate

In [ ]:
def simulate(simulation,i,tsim):
    simulation[i].integr()
    return simulation[i]

In [ ]:
num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sim,i,tsim) for i in range(len(sim)))
sim = sims
print('ready')

### Results

In [ ]:
peak = [sim[i].peak for i in range(len(T_inf))]
peak_t = [sim[i].peak_t for i in range(len(T_inf))]

In [ ]:
for i in range(len(T_inf)):
    #plt.plot(sim[i].t,sim[i].S,label='(S) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].E,label='(E) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,np.log2(sim[i].I),label='(I) T_inf: '+str(T_inf[i]))
    plt.plot(sim[i].t,sim[i].I_d,label='(I_d) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].I,label='(I) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].I_ac,label='(I_ac) T_inf: '+str(T_inf[i]),linestyle='dashed')
    #plt.plot(sim[i].t,sim[i].R,label='(R) T_inf: '+str(T_inf[i]))
plt.title('I vs t: Sigma and Gamma variation')
plt.legend(loc=0)
plt.plot()

In [ ]:
np.cumsum(sim[0].I)

In [ ]:
Iacu = [np.cumsum(sim[i].I) 

In [ ]:
plt.plot(T_inf,peak,label='Peak size vs Infectious period')
plt.title('Peak size vs Infectious period')
plt.legend(loc=0)
plt.plot()

In [ ]:
plt.plot(T_inf,peak_t,label='Peak day')
plt.title('Peak day vs Infectious period')
plt.legend(loc=0)
plt.plot()

In [ ]:
R_final = [sim[i].R[-1] for i in range(len(T_inf))]

In [ ]:
plt.plot(T_inf,R_final,label='R final')
plt.title('Peak day vs Infectious period')
plt.legend(loc=0)
plt.plot(1,1)
plt.plot()

In [ ]:
population

# Estudio tiempo serial para R fijo
Manteniendo el R fijo, variar sigma y gamma. 
Hacer plots de contorno de tamaño y fecha del peak para estas dimensiones

In [ ]:
# Simulation time
tsim = 1000

# Parameters
R0 = 1.2
T_s = 8

step = 0.5
T_inf = np.array(np.arange(1,10+step,step))
T_inc =  np.array(np.arange(1,8+step,step))
gamma = 1/T_inf
sigma = 1/T_inc
beta = R0*gamma 

In [ ]:
#Initial Conditions
mu = R0
population = 100000
I = 100
E = I*mu
R = 0
S = population - I - E - R

In [ ]:
def alpha(t):
    return 1

In [ ]:
i = 0
sim = [[SEIR(tsim=tsim,alpha=alpha,beta=beta[i],mu=mu,sigma=sigma[j],gamma=gamma[i],I0=I,I_ac0=I,I_d0=0,population=population) for j in range(len(T_inc))] for i in range(len(T_inf))]


### Simulate

In [ ]:
def simulate(simulation,i,tsim):
    simulation[i].integr()
    return simulation[i]

In [ ]:
num_cores = multiprocessing.cpu_count()
aux = []
for j in range(len(T_inc)):
    aux.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sim[i],j,tsim) for i in range(len(T_inf))))
#sim = aux
print('ready')

### Results

In [ ]:
sim = aux

In [ ]:
peak = [[sim[i][j].peak for i in range(len(T_inc))] for j in range(len(T_inf))]
peak_t = [[sim[i][j].peak_t for i in range(len(T_inc))] for j in range(len(T_inf))]

In [ ]:
np.shape(peak)

### Variando tiempo de infeccion

In [ ]:
j = 0
for i in range(len(T_inf)):
    #plt.plot(sim[i].t,sim[i].S,label='(S) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].E,label='(E) T_inf: '+str(T_inf[i]))
    plt.plot(sim[j][i].t,sim[j][i].I,label='(I) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].R,label='(R) T_inf: '+str(T_inf[i]))
plt.title('I vs t: T_inc = '+str(T_inc[j]))
plt.legend(loc=0)
plt.plot()

### Variando tiempo de incubacion

In [ ]:
i = 0
for j in range(len(T_inc)):
    #plt.plot(sim[i].t,sim[i].S,label='(S) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].E,label='(E) T_inf: '+str(T_inf[i]))
    plt.plot(sim[j][i].t,sim[j][i].I,label='(I) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].R,label='(R) T_inf: '+str(T_inf[i]))
plt.title('I vs t: T_inf = '+str(T_inf[i]))
plt.legend(loc=0)
plt.plot()

### Contour plots

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(T_inc,T_inf,peak) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Size')
ax.set_xlabel('Incubation Time')
ax.set_ylabel('Infection Time')
plt.show() 

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(T_inc,T_inf,peak_t) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak day')
ax.set_xlabel('Incubation Time')
ax.set_ylabel('Infection Time')
plt.show() 

### Serial Time

In [ ]:
T_s = [[i + j/2 for i in T_inc] for j in T_inf]

In [ ]:
T_s

# Estados iniciales

In [ ]:
# Simulation time
tsim = 1000

# Parameters
R0 = 1.2
T_s = 8

step = 0.5
# gamma
T_inf = 6
# sigma
T_inc = 3
gamma = 1/T_inf
sigma = 1/T_inc

beta = R0*gamma 

In [ ]:
#Initial Conditions
mu = R0
population = 100000
I = np.array(np.arange(1,100,10))
E = I*mu
R = 0
S = population - I - E - R

In [ ]:
def alpha(t):
    return 1

In [ ]:
i = 0
sim = [SEIR(tsim=tsim,alpha=alpha,beta=beta,mu=mu,sigma=sigma,gamma=gamma,I0=I[i],I_ac0=I[i],I_d0=0,population=population) for i in range(len(I))]


### Simulate

In [ ]:
def simulate(simulation,i,tsim):
    simulation[i].integr()
    return simulation[i]

In [ ]:
num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sim,i,tsim) for i in range(len(sim)))
sim = sims
print('ready')

### Results

In [ ]:
peak = [sim[i].peak for i in range(len(I))]
peak_t = [sim[i].peak_t for i in range(len(I))]

In [ ]:
for i in range(len(I)):
    #plt.plot(sim[i].t,sim[i].S,label='(S) T_inf: '+str(T_inf[i]))
    #plt.plot(sim[i].t,sim[i].E,label='(E) T_inf: '+str(T_inf[i]))
    plt.plot(sim[i].t,np.log2(sim[i].I),label='(I)  I0: '+str(I[i]))
    #plt.plot(sim[i].t,sim[i].I,label='(I) I0: '+str(I[i]))
    #plt.plot(sim[i].t,sim[i].R,label='(R) T_inf: '+str(T_inf[i]))
plt.title('I vs t: Sigma and Gamma variation')
plt.legend(loc=0)
plt.plot()

# Ideas Bassi
$I_d(t) = R_e(t)^{\frac{1}{T_S}}$ 